In [59]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_notebook, output_file
import matplotlib as mpl
from matplotlib import cm 
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.palettes import Viridis256 as palette

filename = r"C:\Users\sli\Desktop\Copy of Round2XRD.xlsx"

df= pd.read_excel(filename,index_col='angle')

output_file('XRD.html')
#output_notebook

#x = df.angle
#y = df.iloc[:,1]
DC = [15.9, 15.4, 19.7, 19.4, 19.4, 17.7, 16.4, 16.6, 18.4, 15.6, 9.7, 19.6, 15.0]
colors = [
    "#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b, _ in 255*mpl.cm.viridis(mpl.colors.Normalize()(DC))
]
intpi0=[]
intpi=[]
intpi2=[]
intpi3=[]
make = ['Mazda','Nissan','Honda','Honda','Honda','Honda','Honda','Honda','Honda','Honda','Takata','Takata','Takata']
compart = ['AI','AI','AI','AI','Booster','Booster','Booster','Booster','Booster','Booster','Primary','Primary','AI']

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS, plot_width=1300, plot_height=700, toolbar_location='above')
p.yaxis.axis_label = "Intensity"
p.xaxis.axis_label = "2θ"
p.left[0].formatter.use_scientific = False
p.xaxis.axis_label_text_font_size='2em'
p.yaxis.axis_label_text_font_size='2em'
p.xaxis.major_label_text_font_size='1.5em'
p.yaxis.major_label_text_font_size='1.5em'

color_mapper = LinearColorMapper(palette=palette,low=9.7,high=19.7)

color_bar = ColorBar(color_mapper=color_mapper,location=(0,0),title = "DC (%)", title_text_font_size='1em',major_label_text_font_size='1em')

p.add_layout(color_bar, 'right')

peakdf=df[(df.index<6.6) & (df.index>5.2)]
peak2df=df[(df.index>9.6) & (df.index<10.3)]
peak3df=df[(df.index>11.4) & (df.index<12)]

import scipy
from scipy import integrate

for (name,color,leg) in zip(df,colors,DC):
    p.line(df.index,df[name],legend=str(leg),color=color)
    p.line(df.index,df[name],color=color,alpha=0.6, line_width=2)
    integ=scipy.integrate.trapz(peakdf.loc[:,name],peakdf.index)
    base=scipy.integrate.trapz([peakdf.loc[peakdf.index[0],name],peakdf.loc[peakdf.index[-1],name]],[peakdf.index[0],peakdf.index[-1]])
    intpi.append(integ-base)
    integ2=scipy.integrate.trapz(peak2df.loc[:,name],peak2df.index)
    base2=scipy.integrate.trapz([peak2df.loc[peak2df.index[0],name],peak2df.loc[peak2df.index[-1],name]],[peak2df.index[0],peak2df.index[-1]])
    intpi2.append(integ2-base2)
    integ3=scipy.integrate.trapz(peak3df.loc[:,name],peak3df.index)
    base3=scipy.integrate.trapz([peak3df.loc[peak3df.index[0],name],peak3df.loc[peak3df.index[-1],name]],[peak3df.index[0],peak3df.index[-1]])
    intpi3.append(integ3-base3)
    integ0=scipy.integrate.trapz(df.loc[:,name],df.index)
    base0=scipy.integrate.trapz([df.loc[df.index[0],name],df.loc[df.index[-1],name]],[df.index[0],df.index[-1]])
    #intpi0.append(integ0-base0)
    intpi0.append(integ0)

#show(p)

In [60]:
from bokeh.models import LabelSet, Label,ColumnDataSource
from bokeh.charts import Scatter
from bokeh.layouts import column, gridplot, layout

TOOLS2="hover,crosshair,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,save,box_select,"

infodf=pd.DataFrame([DC,intpi0,intpi,intpi2,intpi3, make,compart]).transpose()
infodf.columns=['dc','int0','int','int2','int3','make','compart']
infodf['cint']=infodf['int'].divide(infodf['int0']).astype('float')
infodf['cint2']=infodf['int2'].divide(infodf['int0']).astype('float')
infodf['cint3']=infodf['int'].divide(infodf['int2']).astype('float')
infodf['cint0']=infodf['int'].divide(infodf['int3']).astype('float')
infodf['cint20']=infodf['int2'].divide(infodf['int3']).astype('float')

infodf.dc=infodf.dc.astype('float')
infodf.int=infodf.int.astype('float')
infodf.int0=infodf.int0.astype('float')
infodf.int2=infodf.int2.astype('float')
infodf.int3=infodf.int3.astype('float')

p3=Scatter(infodf,x='dc',y='int',tools=TOOLS2, color='compart',marker='make',plot_width=600, plot_height=400,title='13X DC vs. XRD Integrated Peak Intensity (111)', xlabel='Dessicant Capacity (%)',ylabel='Integrated Peak Intensity (AU)')
p4=Scatter(infodf,x='dc',y='int2',tools=TOOLS2, color='compart',marker='make',plot_width=600, plot_height=400,title='13X DC vs. XRD Integrated Peak Intensity (220)', xlabel='Dessicant Capacity (%)',ylabel='Integrated Peak Intensity (AU)')
p5=Scatter(infodf,x='dc',y='cint',tools=TOOLS2, color='compart',marker='make',plot_width=600, plot_height=400,title='13X DC vs. XRD Relative Integrated Peak Intensity (111)/total', xlabel='Dessicant Capacity (%)',ylabel='Ratio of Integrated Peak Intensity (AU)')
p6=Scatter(infodf,x='dc',y='cint2',tools=TOOLS2, color='compart',marker='make',plot_width=600, plot_height=400,title='13X DC vs. XRD Relative Integrated Peak Intensity (220)/total', xlabel='Dessicant Capacity (%)',ylabel='Ratio of Integrated Peak Intensity(AU)')
p7=Scatter(infodf,x='dc',y='int3',tools=TOOLS2, color='compart',marker='make',plot_width=600, plot_height=400,title='13X DC vs. XRD Integrated Peak Intensity (311)', xlabel='Dessicant Capacity (%)',ylabel='Integrated Peak Intensity (AU)')
p8=Scatter(infodf,x='dc',y='cint3',tools=TOOLS2, color='compart',marker='make',plot_width=600, plot_height=400,title='13X DC vs. XRD Relative Integrated Peak Intensity (111)/(220)', xlabel='Dessicant Capacity (%)',ylabel='Ratio of Integrated Peak Intensity (AU)')
p0=Scatter(infodf,x='dc',y='int0',tools=TOOLS2, color='compart',marker='make',plot_width=600, plot_height=400,title='13X DC vs. XRD Total Integrated Peak Intensity', xlabel='Dessicant Capacity (%)',ylabel='Total Integrated Peak Intensity (AU)')

p9=Scatter(infodf,x='dc',y='cint',tools=TOOLS2, color='compart',marker='make',plot_width=600, plot_height=400,title='13X DC vs. XRD Relative Integrated Peak Intensity (111)/(311)', xlabel='Dessicant Capacity (%)',ylabel='Ratio of Integrated Peak Intensity (AU)')
p10=Scatter(infodf,x='dc',y='cint2',tools=TOOLS2, color='compart',marker='make',plot_width=600, plot_height=400,title='13X DC vs. XRD Relative Integrated Peak Intensity (220)/(311)', xlabel='Dessicant Capacity (%)',ylabel='Ratio of Integrated Peak Intensity(AU)')


output_file("XRDscatterplot.html")
show(gridplot([[p3,p4],[p5,p6],[p7,p8],[p9,p10],[p0]]))